In [1]:
import json
import inspect
from typing import get_type_hints

class Article:
    pass

class Weather:
    pass

class Directions:
    pass

def calculate_mortgage_payment(loan_amount: int, interest_rate: float, loan_term: int) -> float:
    """Get the monthly mortgage payment given an interest rate percentage."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_article_details(title: str, authors: list[str], short_summary: str, date_published: str, tags: list[str]) -> Article:
    '''Get article details from unstructured article text.
date_published: formatted as "MM/DD/YYYY"'''
    
    # TODO: you must implement this to actually call it later
    pass

def get_weather(zip_code: str) -> Weather:
    """Get the current weather given a zip code."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_directions(start: str, destination: str) -> Directions:
    """Get directions from Google Directions API.
start: start address as a string including zipcode (if any)
destination: end address as a string including zipcode (if any)"""
    
    # TODO: you must implement this to actually call it later
    pass

def get_type_name(t):
    name = str(t)
    if "list" in name or "dict" in name:
        return name
    else:
        return t.__name__

def serialize_function_to_json(func):
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    function_info = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": {
            "type": "object",
            "properties": {}
        },
        "returns": type_hints.get('return', 'void').__name__
    }

    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    return json.dumps(function_info, indent=2)

print(serialize_function_to_json(get_article_details))

{
  "name": "get_article_details",
  "description": "Get article details from unstructured article text.\ndate_published: formatted as \"MM/DD/YYYY\"",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "str"
      },
      "authors": {
        "type": "list[str]"
      },
      "short_summary": {
        "type": "str"
      },
      "date_published": {
        "type": "str"
      },
      "tags": {
        "type": "list[str]"
      }
    }
  },
  "returns": "Article"
}


In [2]:
import xml.etree.ElementTree as ET
import re

def extract_function_calls(completion):
    completion = completion.strip()
    pattern = r"(<multiplefunctions>(.*?)</multiplefunctions>)"
    match = re.search(pattern, completion, re.DOTALL)
    if not match:
        return None
    
    multiplefn = match.group(1)
    root = ET.fromstring(multiplefn)
    functions = root.findall("functioncall")
    return [json.loads(fn.text) for fn in functions]

In [12]:
def generate_hermes_prompt(prompt, functions):
    functions = "\n\n".join([serialize_function_to_json(fn) for fn in functions])
    prompt = f"""<|im_start|>system
You are a helpful assistant with access to the following functions:

{functions}

To use these functions respond with:
<multiplefunctions>
    <functioncall> {{"name": "function_name", "arguments": {{"arg_1": "value_1", "arg_2": value_2, ...}}}} </functioncall>
    <functioncall> {{"name": "function_name", "arguments": {{"arg_1": "value_1", "arg_2": value_2, ...}}}} </functioncall>
    ...
</multiplefunctions>

Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant"""
    return prompt

In [13]:
prompts = [
    "What's the weather in 10001?",
    "Determine the monthly mortgage payment for a loan amount of $200,000, an interest rate of 4%, and a loan term of 30 years.",
    "What's the current exchange rate for USD to EUR?"
]
functions = [get_weather, calculate_mortgage_payment, get_article_details]

for prompt in prompts:
    print(generate_hermes_prompt(prompt, functions))

<|im_start|>system
You are a helpful assistant with access to the following functions:

{
  "name": "get_weather",
  "description": "Get the current weather given a zip code.",
  "parameters": {
    "type": "object",
    "properties": {
      "zip_code": {
        "type": "str"
      }
    }
  },
  "returns": "Weather"
}

{
  "name": "calculate_mortgage_payment",
  "description": "Get the monthly mortgage payment given an interest rate percentage.",
  "parameters": {
    "type": "object",
    "properties": {
      "loan_amount": {
        "type": "int"
      },
      "interest_rate": {
        "type": "float"
      },
      "loan_term": {
        "type": "int"
      }
    }
  },
  "returns": "float"
}

{
  "name": "get_article_details",
  "description": "Get article details from unstructured article text.\ndate_published: formatted as \"MM/DD/YYYY\"",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "str"
      },
      "authors": {
        "

In [5]:
import llama_cpp

llama = llama_cpp.Llama(model_path="../../models/OpenHermes-2.5-Mistral-7B-GGUF/openhermes-2.5-mistral-7b.Q4_K_M.gguf", n_gpu_layers=-1, n_ctx=2048, verbose=False)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 2060, compute capability 7.5
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../../models/OpenHermes-2.5-Mistral-7B-GGUF/openhermes-2.5-mistral-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  

In [22]:
prompts = [
    "What's the weather in 10001?",
    "Determine the monthly mortgage payment for a loan amount of $200,000, an interest rate of 4%, and a loan term of 30 years.",
    "What's the current exchange rate for USD to EUR?"
]
functions = [get_weather, calculate_mortgage_payment, get_article_details]

for prompt in prompts:
    prompt = generate_hermes_prompt(prompt, functions)
    completion = llama.create_completion(prompt, max_tokens=-1)["choices"][0]["text"]
    function_calls = extract_function_calls(completion)
    if function_calls:
        print(function_calls)
    else:
        print(completion.strip())
    print("="*100)

[{'name': 'get_weather', 'arguments': {'zip_code': '10001'}}]
[{'name': 'calculate_mortgage_payment', 'arguments': {'loan_amount': 200000, 'interest_rate': 0.04, 'loan_term': 30}}]
Unfortunately, I do not have a built-in function to check currency exchange rates. However, you can use third-party APIs or websites like Google Finance or XE to get this information.


In [23]:
prompts = [
    "What's the weather in 05751?",
    "I'm planning a trip to Killington, Vermont (05751) from Hoboken, NJ (07030). Can you get me weather for both locations and directions?",
    "What's the current exchange rate for USD to EUR?"
]

for prompt in prompts:
    completion = llama.create_completion(generate_hermes_prompt(prompt, functions), max_tokens=-1)["choices"][0]["text"]
    function_calls = extract_function_calls(completion)

    if function_calls:
        for function in function_calls:
            print(function["name"])
            print(function["arguments"])
    else:
        print(completion.strip())

    print("="*100)

get_weather
{'zip_code': '05751'}
get_weather
{'zip_code': '05751'}
get_weather
{'zip_code': '07030'}
calculate_mortgage_payment
{'loan_amount': 250000, 'interest_rate': 4.18, 'loan_term': 30}
I don't have a function to get exchange rates, but I can provide some resources where you can find this information. You can check websites like Google Finance, XE.com, or Yahoo Finance for up-to-date currency exchange rates.
